# Práctica 1C - Inteligencia Artificial
**Grupo 16**
- Erik Karlgren Domercq
- David Bugoi
- Daniela Alejandra Córdova Porta

## Belén Díaz Agudo -  Facultad de Informática UCM
## Búsqueda local
En esta primera parte usaremos ejercicios paso a paso para familiarizarnos con la resolución de problemas sencillos de optimización, como la maximización o minimización de una función, o el problema de la mochila o del viajante, problemas conocidos cuya resolución se ha abordado con técnicas algorítmicas y que vamos a resolver utilizando algoritmos de búsqueda local. En la segunda parte de la práctica se pide resolver el problema de la organización de jornadas informáticas dado en el enunciado.

In [1]:
cd aima-python

/home/erik/Documents/University/2020-2021/1st semester/IA1/practicas-github/aima-python


## Parte 1. Algoritmo de escalada
Hill Climbing es un algoritmo de búsqueda local heurística utilizada para problemas de optimización.
Esta solución puede o no ser el óptimo global. El algoritmo es una variante del algoritmo de generación y prueba.


En general, el algoritmo funciona de la siguiente manera:
- Evaluar el estado inicial.
- Si es igual al estado del objetivo, terminamos.
- Encuentra un estado vecino al estado actual
- Evaluar este estado. Si está más cerca del estado objetivo que antes, reemplace el estado inicial con este estado y repita estos pasos.


Usaremos la implementación de AIMA que está en el módulo search.py

```python
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state
 ```


### TSP (<i>Travelling Salesman Problem</i>): el problema del viajante
Dado un conjunto de ciudades y la distancia entre cada par de ciudades, el problema es encontrar la ruta más corta posible que visite cada ciudad exactamente una vez y regrese al punto de partida. Es un problema NP hard. No existe una solución de coste polinomial.

In [2]:
## Resolveremos el problema del viajante TSP para encontrar una solución aproximada.
from search import *

class TSP_problem(Problem):

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem """
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self, c, state1, action, state2):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]
        cost += distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return self.path_cost(None, None, None, state)

Resolveremos el TSP para las ciudades de la lista de ciudades de Rumanía.

```python
['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 
 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 
 'Vaslui', 'Zerind']
```

In [3]:
# Usaremos la siguiente representacion del libro AIMA para el mapa de Rumanía.

# Distancias entre las ciudades del mapa
romania_map = UndirectedGraph(dict(
    Arad      = dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest = dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova   = dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta   = dict(Mehadia=75),
    Eforie    = dict(Hirsova=86),
    Fagaras   = dict(Sibiu=99),
    Hirsova   = dict(Urziceni=98),
    Iasi      = dict(Vaslui=92, Neamt=87),
    Lugoj     = dict(Timisoara=111, Mehadia=70),
    Oradea    = dict(Zerind=71, Sibiu=151),
    Pitesti   = dict(Rimnicu=97),
    Rimnicu   = dict(Sibiu=80),
    Urziceni  = dict(Vaslui=142)))

# Coordenadas. Son para renderizar el grafo.
romania_map.locations = dict(
    Arad    =  (91, 492), Bucharest = (400, 327), Craiova  = (253, 288),
    Drobeta = (165, 299), Eforie    = (562, 293), Fagaras  = (305, 449),
    Giurgiu = (375, 270), Hirsova   = (534, 350), Iasi     = (473, 506),
    Lugoj   = (165, 379), Mehadia   = (168, 339), Neamt    = (406, 537),
    Oradea  = (131, 571), Pitesti   = (320, 368), Rimnicu  = (233, 410),
    Sibiu   = (207, 457), Timisoara =  (94, 410), Urziceni = (456, 350),
    Vaslui  = (509, 444), Zerind    = (108, 531))

Es bastante sencillo entender este `romania_map`. El primer nodo **Arad** tiene tres vecinos llamados **Zerind**, **Sibiu**, **Timisoara**. Cada uno de estos nodos son 75, 140, 118 unidades aparte de **Arad** respectivamente. Y lo mismo ocurre con otros nodos.

Y `romania_map.locations` contiene las posiciones de cada uno de los nodos. 
Como heurística se puede usar la distancia en línea recta o la distancia manhattan (que es diferente de la proporcionada en `romania_map`) entre dos ciudades.

In [4]:
romania_locations = romania_map.locations
print(romania_locations)

{'Arad': (91, 492), 'Bucharest': (400, 327), 'Craiova': (253, 288), 'Drobeta': (165, 299), 'Eforie': (562, 293), 'Fagaras': (305, 449), 'Giurgiu': (375, 270), 'Hirsova': (534, 350), 'Iasi': (473, 506), 'Lugoj': (165, 379), 'Mehadia': (168, 339), 'Neamt': (406, 537), 'Oradea': (131, 571), 'Pitesti': (320, 368), 'Rimnicu': (233, 410), 'Sibiu': (207, 457), 'Timisoara': (94, 410), 'Urziceni': (456, 350), 'Vaslui': (509, 444), 'Zerind': (108, 531)}


In [5]:
# node colors, node positions and node label positions
node_colors = {node: 'white' for node in romania_map.locations.keys()}
node_positions = romania_map.locations
node_label_pos = { k:[v[0],v[1]-10]  for k,v in romania_map.locations.items() }
edge_weights = {(k, k2) : v2 for k, v in romania_map.graph_dict.items() for k2, v2 in v.items()}

romania_graph_data = {  'graph_dict' : romania_map.graph_dict,
                        'node_colors': node_colors,
                        'node_positions': node_positions,
                        'node_label_positions': node_label_pos,
                         'edge_weights': edge_weights
                     }

In [6]:
# Descomentar para mostrar el mapa
# from notebook import show_map, final_path_colors, display_visual
# import matplotlib

# show_map(romania_graph_data)

In [7]:
## el siguiente código crea un diccionario y calcula y añade al diccionario la distancia manhattan
## entre las ciudades. 
import numpy as np

distances = {}
all_cities = []

for city in romania_map.locations.keys():
    distances[city] = {}
    all_cities.append(city)
    
all_cities.sort()
print(all_cities)

for name_1, coordinates_1 in romania_map.locations.items():
    for name_2, coordinates_2 in romania_map.locations.items():
        distances[name_1][name_2] = np.linalg.norm(
            [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
        distances[name_2][name_1] = np.linalg.norm(
            [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']


In [8]:
# Creamos una instancia del problema TSP con la lista de ciudades anterior que se na extraido del mapa.
# En el mapa hay informacion de las distancias que se utilizan en la clase TSP_problem para calcular el coste y las heurísticas.
tsp = TSP_problem(all_cities)

In [9]:
## Redefinimos el hill climbing de AIMA para que el método de generacion de vecinos sea acceder al grafo que 
## hemos definido para el TSP

def hill_climbing(problem):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state, number_of_neighbors=100):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors

    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10000
    
    current = Node(problem.initial)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            current.state = neighbor.state
        iterations -= 1
        
    return current.state

In [10]:
# Y lo resolvemos con escalada. 
%time hill_climbing(tsp)

CPU times: user 11 s, sys: 196 ms, total: 11.2 s
Wall time: 11.6 s


['Arad',
 'Bucharest',
 'Craiova',
 'Drobeta',
 'Eforie',
 'Fagaras',
 'Giurgiu',
 'Hirsova',
 'Iasi',
 'Lugoj',
 'Mehadia',
 'Neamt',
 'Oradea',
 'Pitesti',
 'Rimnicu',
 'Sibiu',
 'Timisoara',
 'Urziceni',
 'Vaslui',
 'Zerind']

### Ejercicio 1. Resuelve el problema TSP con el algoritmo de escalada por máxima pendiente en el mapa de ciudades de Rumanía y explica el resultado obtenido. 

**Realiza un análisis razonado de las propiedades del algoritmo: eficiencia y optimalidad en base a la ejecución.**

Este algoritmo encuentra el óptimo local, que puede ser o no el óptimo global, porque solo considera los vecinos en cada estado y coge siempre el mejor. Sin embargo, gracias a ello llega rápidamente a dicho óptimo, que en general es el local, aunque depende del problema.

**¿Ha encontrado el algoritmo el óptimo global? ¿Ha encontrado la misma solución en distintas ejecuciones?
Sólo se pide hacer una comparativa teórica (breve) con cómo se comporta este algoritmo y relacionarlo con otros algoritmos vistos en clase. Opcionalmente se puede hacer la comparativa real con algún algoritmo de búsqueda exhaustiva.**

El algoritmo encuentra normalmente lo que parecen ser óptimos globales con valor 1589.8433308050924, aunque perfectamente podrían ser óptimos locales. Debido al comportamiento aleatorio de `two_opt` de vez en cuando encuentra peores resultados.

Hemos modificado `hill_climbing` para que minimice y `value()` de `TSP_problem` no multiplique por -1.

In [11]:
# hemos obtenido las siguientes soluciones en diferentes ejecuciones
sol1 = ['Arad','Timisoara','Lugoj', 'Mehadia', 'Drobeta', 'Craiova', 'Pitesti', 'Giurgiu', 'Bucharest',
        'Urziceni', 'Eforie', 'Hirsova', 'Vaslui', 'Iasi', 'Neamt', 'Fagaras', 'Rimnicu', 'Sibiu', 'Oradea',
         'Zerind']
sol2 = ['Mehadia', 'Drobeta', 'Craiova', 'Pitesti', 'Giurgiu', 'Bucharest', 'Urziceni', 'Eforie', 'Hirsova',
        'Vaslui', 'Iasi', 'Neamt', 'Fagaras', 'Rimnicu', 'Sibiu', 'Oradea', 'Zerind', 'Arad', 'Timisoara',
        'Lugoj']
sol3 = ['Hirsova', 'Eforie', 'Urziceni', 'Bucharest', 'Giurgiu', 'Craiova', 'Drobeta', 'Mehadia', 'Lugoj',
        'Timisoara', 'Arad', 'Zerind', 'Oradea', 'Sibiu', 'Rimnicu', 'Pitesti', 'Fagaras', 'Neamt', 'Iasi',
        'Vaslui']
sol4 = ['Drobeta', 'Craiova', 'Pitesti', 'Giurgiu', 'Bucharest', 'Urziceni', 'Eforie', 'Hirsova', 'Vaslui',
        'Iasi', 'Neamt', 'Fagaras', 'Rimnicu', 'Sibiu', 'Oradea', 'Zerind', 'Arad', 'Timisoara', 'Lugoj',
        'Mehadia']

print("Valor de sol1:", tsp.value(sol1))
print("Valor de sol2:", tsp.value(sol2))
print("Valor de sol3:", tsp.value(sol3))
print("Valor de sol4:", tsp.value(sol4))

Valor de sol1: 1589.8433308050924
Valor de sol2: 1589.8433308050924
Valor de sol3: 1593.535425958492
Valor de sol4: 1589.8433308050924


Comparamos con algunos algoritmos:
- **Algoritmo de búsqueda por haz local:** este guarda k nodos y utiliza el algoritmo de escalada. Por tanto puede encontrar el óptimo global ya que hay varias opciones. Aun así puede que las k escaladas se queden en óptimos locales.

- **Algoritmo de enfriamiento simulado:** a diferencia de los dos anteriores introduce una variable aleatoria que permite esocger hijos peores evitando así quedarse en un óptimo local. Al principio escogen muchas soluciones peores debido a una alta "temperatura", y al final aceptan pocas soluciones peores debido a una baja "temperatura".

- **Algoritmo de profundidad:** el algoritmo de escalada por la máxima pendiente se parece al de búsqueda de profundidad, pero quitando la parte de volver a estados anteriores. Además, el algoritmo de escalada usa un criterio para elegir por qué hijo avanzar.

## Parte 2. Enfriamiento simulado (<i>simulated annealing</i>) 
El algoritmo de enfriamiento simulado puede manejar las situaciones de óptimo local o mesetas típicas en algoritmos de escalada.
<br>
El enfriamiento simulado es bastante similar a la escalada pero en lugar de elegir el mejor movimiento en cada iteración, elige un movimiento aleatorio. Si este movimiento aleatorio nos acerca al óptimo global, será aceptado,
pero si no lo hace, el algoritmo puede aceptar o rechazar el movimiento en función de una probabilidad dictada por la temperatura.  Cuando la `temperatura` es alta, es más probable que el algoritmo acepte un movimiento aleatorio incluso si es malo. A bajas temperaturas, solo se aceptan buenos movimientos, con alguna excepción ocasional.
Esto permite la exploración del espacio de estado y evita que el algoritmo se atasque en el óptimo local.
    
```python
# Usaremos la implementación de AIMA del modulo search.py

def simulated_annealing(problem, schedule=exp_schedule()):
"""[Figure 4.5] CAUTION: This differs from the pseudocode as it
returns a state instead of a Node."""
current = Node(problem.initial)
for t in range(sys.maxsize):
    T = schedule(t)
    if T == 0:
        return current.state
    neighbors = current.expand(problem)
    if not neighbors:
        return current.state
    next_choice = random.choice(neighbors)
    delta_e = problem.value(next_choice.state) - problem.value(current.state)
    if delta_e > 0 or probability(math.exp(delta_e / T)):
        current = next_choice`
 ```

Como hemos visto en clase hay varios métodos de enfriamiento (scheduling routine) 
Se puede variar el método de enfriamiento. En la implementación actual estamos usando el método de enfriamiento exponencial (que se pasa como parámetro). 
```python
def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)
```

Como ejemplo, vamos a definir un problema sencillo de encontrar el punto más alto en una rejilla. Este problema está definido en el módulo search.py como `PeakFindingProblem`. Lo reproducimos aquí y creamos una rejilla simple.

In [12]:
initial = (0, 0)
grid = [[3, 7, 2, 8],
        [5, 2, 9, 1],
        [5, 3, 3, 1]]

In [13]:
# Pre-defined actions for PeakFindingProblem
directions4 = { 'W':(-1, 0), 'N':(0, 1), 'E':(1, 0), 'S':(0, -1) }
directions8 = dict(directions4) 
directions8.update({'NW':(-1, 1), 'NE':(1, 1), 'SE':(1, -1), 'SW':(-1, -1) })

class PeakFindingProblem(Problem):
    """Problem of finding the highest peak in a limited grid"""

    def __init__(self, initial, grid, defined_actions=directions4):
        """The grid is a 2 dimensional array/list whose state is specified by tuple of indices"""
        Problem.__init__(self, initial)
        self.grid = grid
        self.defined_actions = defined_actions
        self.n = len(grid)
        assert self.n > 0
        self.m = len(grid[0])
        assert self.m > 0

    def actions(self, state):
        """Returns the list of actions which are allowed to be taken from the given state"""
        allowed_actions = []
        for action in self.defined_actions:
            next_state = vector_add(state, self.defined_actions[action])
            if next_state[0] >= 0 and next_state[1] >= 0 and next_state[0] <= self.n - 1 and next_state[1] <= self.m - 1:
                allowed_actions.append(action)

        return allowed_actions

    def result(self, state, action):
        """Moves in the direction specified by action"""
        return vector_add(state, self.defined_actions[action])

    def value(self, state):
        """Value of a state is the value it is the index to"""
        x, y = state
        assert 0 <= x < self.n
        assert 0 <= y < self.m
        return self.grid[x][y]


In [14]:
problem = PeakFindingProblem(initial, grid, directions4)

In [15]:
# Lo resolvemos con enfriamiento simulado

solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

9

In [16]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [17]:
solution = problem.value(hill_climbing(problem))
solution

7

### Ejercicio 2.  Resuelve el problema anterior de encontrar el punto máximo en una rejilla. Comenta y razona los resultados obtenidos en distintas rejjillas con los algoritmos de enfriamiento simulado y escalada por máxima pendiente. 

In [27]:
# Ejemplo de rejilla para pruebas
grid2 = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
         [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
         [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
         [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
         [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
         [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
         [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]
grid3 = [[3,2,3],
         [2,5,6],
         [7,8,9]]

In [30]:
# Resuelve un problema ejecutando 100 veces el algoritmo de enfriamiento y 1 el de escalada e
# imprime el resultado de aplicar cada uno.
# Ejecutar varias veces el algoritmo de escalada daría siempre el mismo resultado, pero no pasa igual
# con el de enfriamiento.
def soluciones_enfriamiento_y_escalada(problem):
    sol_sim  = max({problem.value(simulated_annealing(problem)) for i in range(100)})
    sol_hill = problem.value(hill_climbing(problem))
    print("Solución por enfriamiento:", sol_sim)
    print("Solución por escalada:", sol_hill)
    print()

In [38]:
soluciones_enfriamiento_y_escalada(PeakFindingProblem((0,0), grid2, directions4))  # 1
soluciones_enfriamiento_y_escalada(PeakFindingProblem((1,5), grid2, directions4))  # 2
soluciones_enfriamiento_y_escalada(PeakFindingProblem((1,5), grid2, directions8))  # 3
soluciones_enfriamiento_y_escalada(PeakFindingProblem((7,7), grid2, directions4))  # 4
soluciones_enfriamiento_y_escalada(PeakFindingProblem((0,0), grid3, directions4))  # 5

Solución por enfriamiento: 9.0
Solución por escalada: 0.0

Solución por enfriamiento: 11.2
Solución por escalada: 0.0

Solución por enfriamiento: 11.2
Solución por escalada: 0.4

Solución por enfriamiento: 11.2
Solución por escalada: 11.2

Solución por enfriamiento: 9
Solución por escalada: 3



El algoritmo de enfriamiento consigue consistentemente mejores resultados que el algoritmo de escalada. Esto se debe a que el de escalada simple coge cualquier estado vecino que sea mejor que el estado actual, pero el de enfriamiento a veces coge estados peores, con lo que así se exploran muchas más posibilidades.

Vemos en todos los ejemplos de arriba (salvo el 4) que el algoritmo de escalada se queda atascado en un óptimo local, mientras que el de enfriamiento llega a explorar otras soluciones y por ello, tras 100 ejecuciones, al menos una de dichas soluciones llega al óptimo global (no tendría por qué ocurrir pero sería ínfimamente probable).

Justo en el ejemplo 4 el algoritmo de escalada simple llega también al óptimo global por ser el óptimo local del punto de partida.

## Parte 3. Algoritmos genéticos


Se define una clase `ProblemaGenetico` que incluye los elementos necesarios para la representación de un problema de optimización que se va a resolver con un algoritmo genético. Los elementos son los que hemos visto en clase:

 - `genes`: lista de genes usados en el genotipo de los estados.
 - `longitud_individuos`: longitud de los cromosomas
 - `decodifica`: función que obtiene el fenotipo a partir del genotipo.
 - `fitness`: función de valoración.
 - `muta`: función de mutación de un cromosoma 
 - `cruza`: función de cruce de un par de cromosomas

In [26]:
import random

In [27]:
class ProblemaGenetico(object):
        def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness,longitud_individuos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce
        
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma)
            return valoracion

En primer lugar vamos a definir una instancia de la clase anterior correspondiente al problema de optimizar (maximizar o minimizar) la función cuadrado x<sup>2</sup> en el conjunto de los números naturales menores que 2<sup>10</sup>.
Se usa este ejemplo (del que sabemos la solución) para ver todos los elementos y poder observar el comportamiento. 


In [28]:
# Será necesaria la siguiente función que interpreta una lista de 0's y 1's como un número natural:  
# La siguiente función que interpreta una lista de 0's y 1's como
# un número natural:  

def binario_a_decimal(x):
    return sum(b*(2**i) for (i,b) in enumerate(x)) 

In [29]:
list(enumerate([1, 0, 0]))

[(0, 1), (1, 0), (2, 0)]

In [30]:
# En primer lugar usaremos la clase anterior para representar el problema de optimizar (maximizar o minimizar)
# la función cuadrado en el conjunto de los números naturales menores que
# 2^{10}.

# Tenemos que definir funciones de cruce, mutación y fitness para este problema.

def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2]+cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2]+cromosoma1[l2//2:l1]
    return [cruce1,cruce2]

def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        cromosoma[p] =  (cromosoma[p]+1)%2
    return cromosoma

def fun_fitness_cuad(cromosoma):
    """Función de valoración que eleva al cuadrado el número recibido en binario"""
    n = binario_a_decimal(cromosoma)**2
    return n

cuadrados = ProblemaGenetico([0,1],binario_a_decimal,fun_mutar, fun_cruzar, fun_fitness_cuad,10)

Una vez definida la instancia cuadrados que representa el problema genético, probar alguna de las funciones definidas en la clase anterior, para esta instancia concreta. Por ejemplo:

In [31]:
cuadrados.decodifica([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1329

1329

In [32]:
cuadrados.fitness([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1766241

1766241

In [33]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [34]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1]

In [35]:
cuadrados.cruza([1,0,0,0,1,1,0,0,1,0,1],[0,1,1,0,1,0,0,1,1,1])
# Posible salida: [[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

[[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

### Ejercicio 3

   - Definir una función `poblacion_inicial(problema_genetico,tamaño)`, para definir una población inicial de un tamaño dado, para una instancia dada de la clase anterior `ProblemaGenetico`

Sugerencia: usar `random.choice`

   - Definir una función de cruce que recibe una instancia de `Problema_Genetico` y una población de padres (supondremos que hay un número par de padres), obtiene la población resultante de cruzarlos de dos en dos (en el orden en que aparecen)

`cruza_padres(problema_genetico,padres)`

   - Definir la función de mutación que recibe una instancia de `Problema_Genetico`, una población y una probabilidad de mutación, obtiene la población resultante de aplicar operaciones de mutación a cada individuo llamando a la función muta definida para el problema genético.

`muta_individuos(problema_genetico, poblacion, prob)`

In [36]:
def poblacion_inicial(problema_genetico, size):
    l = []
    for i in range(size):
        l.append([random.choice(problema_genetico.genes) for i in range(problema_genetico.longitud_individuos)])
    return l

In [37]:
poblacion_inicial(cuadrados,10)

[[0, 0, 0, 0, 1, 0, 1, 1, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
 [1, 1, 1, 0, 1, 1, 0, 0, 1, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 [1, 0, 0, 1, 1, 0, 1, 1, 0, 1]]

In [38]:
def cruza_padres(problema_genetico,padres):
    l = []
    l1 = len(padres)
    while padres != []:
        l.extend(problema_genetico.cruza(padres[0],padres[1]))
        padres.pop(0)
        padres.pop(0)
    return l

In [39]:
p = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
     [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
     [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
     [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
     [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
     [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

cruza_padres(cuadrados,p)
# Posible salida
# [[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
#  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#  [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
#  [0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
#  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]]

[[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

In [40]:
# hay que llamar a  problema_genetico.muta(x,prob) para todos los individuos de la poblacion.
def muta_individuos(problema_genetico, poblacion, prob):
    for x in poblacion:
        problema_genetico.muta(x,prob)
    return poblacion
    

In [41]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

muta_individuos(cuadrados,p1,0.5)
# Posible salida:
#  [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#   [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
#   [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
#   [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#   [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

[[1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [42]:
p2 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [43]:
muta_individuos(cuadrados,p2,0.5)

[[1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

### Ejercicio 4

Se pide definir una función de selección mediante torneo de n individuos de una población.  
La función recibe como entrada:
 - Una instancia de la clase `ProblemaGenetico`
 - Una población
 - El número n de individuos que vamos a seleccionar
 - El número k de participantes en el torneo
 - Un valor `opt` que puede ser o la función `max` o la función `min` (dependiendo de si el problema es de maximización o de minimización, resp.).

`seleccion_por_torneo(problema_genetico,poblacion,n,k,opt)`

**Indicación:** Usar `random.sample` para seleccionar k elementos de una secuencia. 
Por ejemplo, `random.sample(population=[2,5,7,8,9], k=3)` devuelve `[7,5,8]`. 

In [97]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

In [98]:
#Ejemplo
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,max)
# Posible salida: [[1, 1, 1, 1, 1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]]

[[1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1]]

In [99]:
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,min)
# [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 1, 0, 1, 0]]

[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0]]

In [182]:
# La siguiente función implementa una posibilidad para el algoritmo genético completo: 
# inicializa t = 0 
# Generar y evaluar la Población P(t)
# Mientras no hemos llegado al número de generaciones fijado:  t < nGen
#    P1 = Selección por torneo de (1-size)·p individuos de P(t)
#    P2 = Selección por torneo de (size·p) individuos de P(t)
#    Aplicar cruce en la población P2
#    P4 = Union de P1 y P3
#    P(t+1) := Aplicar mutación P4 
#    Evalua la población P(t+1) 
#    t:= t+1
        
# Sus argumentos son:
# problema_genetico: una instancia de la clase ProblemaGenetico con la representación adecuada del problema de optimización 
# que se quiere resolver.
#
# k:             número de participantes en los torneos de selección.
# opt:           max ó min, dependiendo si el problema es de maximización o de minimización. 
# nGen:          número de generaciones (que se usa como condición de terminación)
# size:          número de individuos en cada generación
# prop_cruce:    proporción del total de la población que serán padres. 
# prob_mutación: probabilidad de realizar una mutación de un gen.

def algoritmo_genetico(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 


Necesitarás definir la función auxiliar `nueva_generacion(problema_genetico,poblacion,n_padres,n_directos,prob_mutar)` que dada una población calcula la siguiente generación.

In [183]:
#Definir la función nueva_generacion
def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

### Ejercicio 5.  Ejecutar el algoritmo genético anterior, para resolver el problema anterior (tanto en minimización como en maximización).  

Hacer una valoración de resultados y comentarios sobre el comportamiento del algoritmmo. 
En la resolución del problema hay que tener en cuenta que el algoritmo genético devuelve un par con el mejor fenotipo encontrado y su valoración.

In [102]:
algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1)
# Salida esperada: (0, 0)

(136, 18496)

In [103]:
algoritmo_genetico(cuadrados,3,max,20,10,0.7,0.1)
# Salida esperada: (1023, 1046529)

(1022, 1044484)

***COMENTAR***

##  El problema de la mochila 

Se plantea el típico problema de la mochila en el que dados n objetos de pesos conocidos p<sub>i</sub> y valor v<sub>i</sub> (i=1, ..., n) hay que elegir cuáles se meten en una mochila que soporta un peso P máximo. La selección debe hacerse de forma que se máximice el valor de los objetos introducidos sin superar el peso máximo.

### Ejercicio 6
Se pide definir la representación del problema de la mochila usando genes `[0,1]` y longitud de los individuos `n`.

Los valores 1 ó 0 representan, respectivamente, si el objeto se introduce o no en la mochila. Tomados de izquerda a derecha, a partir del primero que no cabe, se consideran  todos fuera de la mochila, independientemente del gen en su posición. De esta manera, todos los individuos representan candidatos válidos.

El numero de objetos n determina la longitud de los individuos de la población.
En primer lugar es necesario definir una función de decodificación de la mochila que recibe como entrada:
* Un `cromosoma` (en este caso, una lista de 0s y 1s, de longitud igual a `n_objetos`) 
* `n`: número total de objetos de la mochila
* `pesos`: una lista con los pesos de los objetos
* `capacidad`: peso máximo de la mochila.
La función decodifica recibe `(cromosoma, n, pesos, capacidad)` y devuelve una lista de 0s y 1s que indique qué objetos están en la mochila y cuáles no (el objeto i está en la mochila si y sólo si en la posición i-ésima de la lista hay un 1). Esta lista se obtendrá a partir del cromosoma, pero teniendo en cuenta que a partir del primer objeto que no quepa, éste y los siguientes se consideran fuera de la mochila, independientemente del valor que haya en su correspondiente posición de cromosoma. 

In [104]:
def decodifica(cromosoma, n, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i]== 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l 

In [105]:
decodifica([1,1,1,1,1], 5, [2,3,4,5,1], 5)

[1, 1, 0, 0, 0]

In [106]:
decodifica([1,0,1,1,1], 5, [2,3,4,5,1], 5)

[1, 0, 0, 0, 1]

Para definir la función de evaluación (`fitness`) necesitamos calcular el valor total de los objetos que están dentro de la mochila que representa el cromosoma según la codificación utilizada en la función anterior. 

Se pide la función `fitness (cromosoma, n_objetos, pesos, capacidad, valores)`, donde los parámetros son los mismos que en la función anterior, y valores es la lista de los valores de cada objeto

`fitness(cromosoma, n_objetos, pesos, capacidad, valores)`

Ejemplo de uso:
```python
fitness([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])
7
```

In [107]:
def fitness_mochila(cromosoma, n_objetos, pesos, capacidad, valores):
    objetos = decodifica(cromosoma, n_objetos, pesos, capacidad)
    valor = 0
    for i in range(n_objetos):
        if objetos[i] == 1:
            valor += valores[i]
    return valor

In [108]:
fitness_mochila([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])

7

Damos tres instancias concretas del problema de la mochila. Damos también sus soluciones optimas, para que se puedan comparar con los resultados obtenidos por el algoritmo genético:

In [109]:
# Problema de la mochila 1:
# 10 objetos, peso máximo 165
pesos1   = [23,31,29,44,53,38,63,85,89,82]
valores1 = [92,57,49,68,60,43,67,84,87,72]

# Solución óptima = [1,1,1,1,0,1,0,0,0,0], con valor 309

In [110]:
fitness_mochila([1,1,1,1,0,1,0,0,0,0], 10, pesos1, 165, valores1)

309

In [111]:
# Problema de la mochila 2:
# 15 objetos, peso máximo 750

pesos2   = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
valores2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

# Solución óptima= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1] con valor 1458

In [112]:
fitness_mochila([1,0,1,0,1,0,1,1,1,0,0,0,0,1,1], 15, pesos2, 750, valores2)

1458

In [113]:
# Problema de la mochila 3:
# 24 objetos, peso máximo 6404180
pesos3 = [382745, 799601, 909247, 729069, 467902,  44328,  34610, 698150,
          823460, 903959, 853665, 551830, 610856, 670702, 488960, 951111,
          323046, 446298, 931161,  31385, 496951, 264724, 224916, 169684]
valores3 = [ 825594, 1677009, 1676628, 1523970,  943972,   97426,  69666, 1296457,
            1679693, 1902996, 1844992, 1049289, 1252836, 1319836, 953277, 2067538,
            675367,   853655, 1826027,   65731,  901489,  577243, 466257, 369261]

# Solución óptima= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1] con valoración 13549094

In [114]:
fitness_mochila([1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1], 24, pesos3, 6404180, valores3)

13549094

### Ejercicio 7

Definir variables `m1g`, `m2g` y `m3g`, referenciando a instancias de `Problema_Genetico` que correspondan, respectivamente, a los problemas de la mochila anteriores. Resuelve los problemas y comentar los resultados obtenidos en cuanto a eficiencia y calidad de los resultados obtenidos.

Algunas de las salidas posibles variando los parámetros.

In [118]:

def fitness_mochila_1(cromosoma):
    v = fitness_mochila(cromosoma, 10, pesos1, 165, valores1)
    return v
def decodifica_mochila_1(cromosoma):
    v = decodifica(cromosoma, 10, pesos1, 165)
    return v
m1g = ProblemaGenetico([0,1], decodifica_mochila_1, fun_mutar, fun_cruzar, fitness_mochila_1,10)

def fitness_mochila_2(cromosoma):
    v = fitness_mochila(cromosoma, 15, pesos2, 750, valores2)
    return v
def decodifica_mochila_2(cromosoma):
    v = decodifica(cromosoma, 14, pesos2, 750)
    return v
m2g = ProblemaGenetico([0,1], decodifica_mochila_2, fun_mutar, fun_cruzar, fitness_mochila_2,15)

def fitness_mochila_3(cromosoma):
    v = fitness_mochila(cromosoma, 24, pesos3,6404180 , valores3)
    return v
def decodifica_mochila_3(cromosoma):
    v = decodifica(cromosoma, 24, pesos3, 6404180)
    return v
m3g = ProblemaGenetico([0,1], decodifica_mochila_3, fun_mutar, fun_cruzar, fitness_mochila_3,24)


**idiota**
```
# def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness,longitud_individuos):
def fit1g(cromosoma):
    return fitness_mochila(cromosoma, pesos=pesos1, n_objetos=10, capacidad=165, valores=valores1)

def decodifica1g(cromosoma):
    return decodifica(cromosoma, n=10, pesos=pesos1, capacidad=165)

m1g = ProblemaGenetico([0,1], decodifica1g, fun_mutar, fun_cruzar, fit1g, 10)
algoritmo_genetico(m1g,3,max,100,50,0.8,0.05)
```

In [119]:
algoritmo_genetico(m1g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 1, 0, 1, 0, 0, 0, 0], 309)

# >>> algoritmo_genetico_t(m2g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0], 1444)
# >>> algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0], 1439)
# >>> algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1], 1458)

# >>> algoritmo_genetico_t(m3g,5,max,400,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], 13518963)
# >>> algoritmo_genetico_t(m3g,4,max,600,200,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 13524340)
# >>> algoritmo_genetico_t(m3g,4,max,1000,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13449995)
# >>> algoritmo_genetico_t(m3g,3,max,1000,100,0.75,0.1)
# ([1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13412953)
# >>> algoritmo_genetico_t(m3g,3,max,2000,100,0.75,0.1)
# ([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 13366296)
# >>> algoritmo_genetico_t(m3g,6,max,2000,100,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1], 13549094)

TypeError: 'NoneType' object is not iterable

In [67]:
algoritmo_genetico(m3g,5,max,400,200,0.75,0.1)

([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1],
 13477838)

### Ejercicio 8
Resolver mediante una configuración de un algoritmo genético el problema del cuadrado mágico que consiste en colocar en un cuadrado n × n los números naturales de 1 a n<sup>2</sup>, 
de tal manera que las filas, las columnas y las diagonales principales sumen lo mismo.

Ejemplo: una solucion para n = 3
    
    4 3 8
    9 5 1
    2 7 6
    
- Dimensión del cuadrado: N
- Suma común: SUMA = (N (N<sup>2</sup> + 1)) / 2
    
**Comenta el resultado y el rendimiento del algoritmo para distintos parámetros:**
    
***COMENTAR***

In [190]:
def suma_comun(n):
    return (n * (n*n +1))//2

# lista es una lista de n^2 elementos -> se puede hacer un cuadrado
def matriz_por_filas(lista, n):
    mat = list()
    for i in range(0, n):
        fila = [lista[x + i*n] for x in range(0,n)]
        mat.append(fila)
    return mat


# lista es una lista de n^2 elementos -> se puede hacer un cuadrado
def matriz_por_cols(lista, n):
    mat = list()
    for i in range(0, n):
        col = [lista[x*n + i] for x in range(0,n)]
        mat.append(col)
    return mat

# cromosoma: lista de números de 1 a n^2
def fitness_cuadrado(cromosoma, n):
    valor = 0
    matriz_filas = matriz_por_filas(cromosoma, n)
    matriz_cols = matriz_por_cols(cromosoma, n)
    suma_a_llegar = suma_comun(n)
    suma_diag1 = 0
    suma_diag2 = 0

    for i in range(0, n):
        # suma filas
        if sum(matriz_filas[i]) == suma_a_llegar:
            valor += 1

        # suma columnas
        if sum(matriz_cols[i]) == suma_a_llegar:
            valor += 1

        # vamos sumando valores en la diagonal
        suma_diag1 += matriz_filas[i][i]
        suma_diag2 += matriz_filas[i][n - i - 1]

    if suma_diag1 == suma_a_llegar:
        valor += 1

    if suma_diag2 == suma_a_llegar:
        valor += 1

    if valor != 8:  ##Penaliza cromosomas con alelos cuyas sumas sean muy altas
        if valoresAltosCercanos(cromosoma, matriz_filas, matriz_cols, n) > 0:
            valor -= 1
        if valoresBajosCercanos(cromosoma, matriz_filas, matriz_cols, n) > 0:
            valor -= 1

    return valor


def valoresAltosCercanos(cromosoma, matrizFilas, matriz_cols, n):
    cont=0
    valorAlto=n*n
    evitable=0
    suma_diag1=0
    suma_diag2=0
    for i in range(0, n):
        evitable +=valorAlto
        valorAlto-=3

    for i in range(0, n):
        suma_diag1 += matrizFilas[i][i]
        suma_diag2 += matrizFilas[i][n - i - 1]
        if (sum(matrizFilas[i])>= evitable):
            cont += 1
        if (sum(matriz_cols[i]) >= evitable):
            cont += 1

    if (suma_diag1>=evitable or suma_diag2>=evitable):
        cont+=1

    return cont

def valoresBajosCercanos(cromosoma, matrizFilas, matriz_cols, n):
    cont=0
    valorAlto=n*n
    evitable=0
    suma_diag1=0
    suma_diag2=0
    for i in range(0, n):
        evitable +=valorAlto
        valorAlto+=3

    for i in range(0, n):
        suma_diag1 += matrizFilas[i][i]
        suma_diag2 += matrizFilas[i][n - i - 1]
        if (sum(matrizFilas[i])<= evitable):
            cont += 1
        if (sum(matriz_cols[i]) >= evitable):
            cont += 1
    if (suma_diag1>=evitable or suma_diag2>=evitable):
        cont+=1
    return cont

def mutar_cuad(cromosoma, n, prob):
    índiceA = random.randint(0, n * n - 1)
    índiceB = random.randint(0, n * n - 1)
    if prob > random.uniform(0, 1):
        cromosoma[índiceA], cromosoma[índiceB] = cromosoma[índiceB], cromosoma[índiceA]
    return cromosoma
# cruce pmx


def pmx_aux(a,b, start, stop):
    child = [None]*len(a)
    # Copy a slice from first parent:
    child[start:stop] = a[start:stop]
    # Map the same slice in parent b to child using indices from parent a:
    for ind,x in enumerate(b[start:stop]):
        ind += start
        if x not in child:
            while child[ind] != None:
                ind = b.index(a[ind])
            child[ind] = x
    # Copy over the rest from parent b
    for ind,x in enumerate(child):
        if x == None:
            child[ind] = b[ind]
    return child

def pmx(a,b):
    half = len(a) // 2
    start = random.randint(0, len(a)-half)
    stop = start + half
    return pmx_aux(a,b,start,stop) , pmx_aux(b,a,start,stop)



In [191]:
fitness_cuadrado([4,3,8,9,5,1,2,7,6], 3)

8

In [192]:
pmx([1,2,3,4,5,6,7,8,9],[9,8,7,6,5,4,3,2,1])

([9, 8, 3, 4, 5, 6, 7, 2, 1], [1, 2, 7, 6, 5, 4, 3, 8, 9])

In [193]:
matriz_por_filas([4,3,8,9,5,1,2,7,6], 3)

[[4, 3, 8], [9, 5, 1], [2, 7, 6]]

In [194]:
matriz_por_cols([x for x in range(1,10)], 3)

[[1, 4, 7], [2, 5, 8], [3, 6, 9]]

In [195]:
fitness_cuadrado([4,3,8,9,5,1,2,7,6], 3)

8

In [196]:
n1 = 3
l = [x for x in range(1, n1*n1 + 1)]
def fit1_cuad(cromosoma):
    return fitness_cuadrado(cromosoma, n1)
def mutar1_cuad(cromosoma, prob):
    return mutar_cuad(cromosoma,n1, prob)
def decod1_cuad(cromosoma):
    return matriz_por_filas(cromosoma,n1)
def cruzar_cuad(cromosoma1, cromosoma2):
    return pmx(cromosoma1, cromosoma2)


In [197]:
len(l)

9

In [198]:
pCuad1 = ProblemaGenetico(l, decod1_cuad, mutar1_cuad, cruzar_cuad,fit1_cuad,n1*n1)

In [199]:
def poblacion_inicial_cuad(problema_genetico, size):
    l = []
    for i in range(0,size):
        l.append(i+1)
    for i in range(size):
        num1=random.randint(0,size-1)
        num2=random.randint(0,size-1)
        a = l [num1]
        b = l [num2]
        l [num1]=b
        l [num2]=a
    return l

In [200]:
poblacion_inicial_cuad(pCuad1,9)

[1, 2, 3, 5, 6, 4, 7, 9, 8]

In [201]:
def poblacion_inicial(problema_genetico, size):
    l = []
    for i in range(size):
        l.append(poblacion_inicial_cuad(problema_genetico, problema_genetico.longitud_individuos))
    return l

In [202]:
poblacion_inicial(pCuad1,10)

[[2, 8, 4, 1, 5, 6, 7, 3, 9],
 [6, 9, 2, 7, 4, 3, 8, 5, 1],
 [4, 2, 8, 1, 5, 3, 7, 9, 6],
 [5, 9, 4, 8, 1, 2, 7, 6, 3],
 [5, 7, 3, 6, 4, 8, 9, 1, 2],
 [2, 6, 1, 3, 5, 8, 4, 7, 9],
 [8, 5, 7, 4, 1, 2, 6, 9, 3],
 [5, 1, 6, 4, 3, 2, 8, 7, 9],
 [5, 7, 4, 6, 3, 2, 1, 8, 9],
 [1, 3, 4, 9, 7, 2, 5, 8, 6]]

In [203]:
algoritmo_genetico(pCuad1, 9, max, 100, 50,0.7,0.1 )

([[7, 2, 6], [3, 4, 8], [5, 9, 1]], 6)

In [204]:
algoritmo_genetico(pCuad1, 9, max, 100, 50,0.7,0.1 )

([[6, 1, 8], [7, 5, 3], [2, 9, 4]], 8)

In [206]:
algoritmo_genetico(pCuad1, 9, max, 100, 50,0.7,0.1 )

([[8, 3, 4], [1, 5, 9], [6, 7, 2]], 8)

In [210]:
algoritmo_genetico(pCuad1, 9, max, 100, 50,0.7,0.1 )

([[5, 3, 7], [9, 4, 2], [1, 8, 6]], 6)

In [225]:
algoritmo_genetico(pCuad1, 9, max, 300, 50,0.7,0.1 )

([[1, 7, 2], [6, 5, 4], [8, 3, 9]], 4)

In [223]:
algoritmo_genetico(pCuad1, 10, max, 100, 70,0.8,0.1 )

([[8, 4, 3], [1, 5, 9], [7, 6, 2]], 5)

### Ejercicio 9
Se quiere organizar una **jornada de la informática** en la Facultad. 
El Vicedecano necesita organizar el horario y la asignación de profesores para n actividades temáticas, a<sub>1</sub>, ... , a<sub>n</sub>.  Dispone de m profesores p<sub>1</sub>, . . . , p<sub>m</sub> con distintos niveles de cualificación.

No todos los profesores tienen los conocimientos para encargarse de todas las actividades. 
El mismo profesor p<sub>i</sub> puede encargarse de distintas actividades (que se le remunerarán convenientemente) pero no podemos asignar al mismo profesor dos actividades que se impartan simultáneamente. 
Cada actividad puede durar un tiempo diferente dependiendo del profesor que la organice. Si un profesor p<sub>i</sub> puede realizar la actividad a<sub>j</sub>, entonces denotaremos por t<sub>ij</sub> el tiempo que p<sub>i</sub> tardará en realizar la actividad a<sub>j</sub>.

Existen dependencias entre los contenidos de las actividades lo que obliga a incluir algunas restricciones temporales, es decir, algunas actividades hay que realizarlas cuando otras ya hayan terminado. Esto es, tenemos un conjunto de restricciones que indican que a<sub>i</sub> se debe realizar antes de a<sub>j</sub>. Por ejemplo, _El taller de iniciación a Scratch_ se debe realizar antes que el laboratorio de desarrollo de videojuegos con Scratch.

Se pide dar una solución al problema de decidir, para cada actividad, qué profesor la realizará y en qué momento se debe ofertar la actividad para que, al ﬁnal de la jornada, se puedan realizar todas las actividades con cuidado de cumplir los prerrequisitos (restricciones temporales) y el conjunto de actividades se realice en el menor tiempo posible.